In [3]:
from __future__ import print_function
import argparse
import os
import sys
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
cudnn.fastest = True
import torch.optim as optim
import torchvision.utils as vutils
from torch.autograd import Variable
import numpy as np
from torchsummary import summary
import datetime
from random import randrange, uniform
import matplotlib.pyplot as plt
from matplotlib import cm

from data_load_gan import index_group, data_loading
from gan_models import D, G

In [7]:
# define the GPU environment
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
# load the area by different classes
channel_map = np.load('/home/gtx1080/Abduallah/pix2pix.pytorch/channel_map.npy')
channel_map = torch.tensor(channel_map, dtype = torch.float, requires_grad = False)
# load the path to training and test data
path_train_data = '../../Sync/Kun/30_min_ele/non_ele_whole/train/'
path_test_data = '../../Sync/Kun/30_min_ele/non_ele_whole/test/'
# path to save trained models
path_save = '../../Sync/Kun/0718_newGAN/'

# define training details
manualSeed = 101
inputChannelSize = 5
outputChannelSize = 3
batchSize = 64
imageSize = 100
# valBatchSize = 
print_cost = True
# variables that uncertain about
ngf = 64
ndf = 64
lambdaGAN = 1
lambdaIMG = 200
lambdaIMGp = 100
lrD = 0.001
lrG = 0.001
niter = 1
train_num = 10000
test_num = 2000
latent_dim = 512
# record the cost after every epoch
costs = []
test_costs = []

# evalIter = 

# control the random seed
random.seed(manualSeed)
torch.manual_seed(manualSeed)
# load the model
netG = G()
# if opt.netG != '':
#   netG.load_state_dict(torch.load(opt.netG))
# print(netG)
netD = D(inputChannelSize + outputChannelSize, ndf)
# if opt.netD != '':
#   netD.load_state_dict(torch.load(opt.netD))
# print(netD)

netG.train()
netD.train()

# define the loss functions
criterionBCE = nn.BCELoss()
criterionCAE = nn.L1Loss()
# CAEP is to add more weights to the channel part 
criterionCAEP = nn.L1Loss()

target_ = torch.FloatTensor(batchSize, outputChannelSize, imageSize, imageSize)
input_ = torch.FloatTensor(batchSize, inputChannelSize, imageSize, imageSize)
input_noise = torch.FloatTensor(batchSize, inputChannelSize, imageSize, imageSize)
latent_noise = torch.FloatTensor(batchSize, latent_dim, 1, 1)

label_d = torch.FloatTensor(batchSize)

# NOTE: size of 2D output maps in the discriminator
sizePatchGAN = 10
real_label = 1
fake_label = 0

# NOTE weight for L_cGAN and L_L1 (i.e. Eq.(4) in the paper)
netD.cuda()
netG.cuda()
criterionBCE.cuda()
criterionCAE.cuda()
criterionCAEP.cuda()

target, input_, label_d = target_.cuda(), input_.cuda(), label_d.cuda()
latent_noise = latent_noise.cuda()
input_noise = input_noise.cuda()

target = Variable(target)
input_ = Variable(input_)
input_noise = Variable(input_noise)
latent_noise - Variable(latent_noise)
label_d = Variable(label_d)


# define the optimizer 
optimizerD = optim.Adam(netD.parameters(), lr = lrD, betas = (0.9, 0.999), weight_decay=0.0)
optimizerG = optim.Adam(netG.parameters(), lr = lrG, betas = (0.9, 0.999), weight_decay=0.0)
channel_map = channel_map.cuda()

# NOTE training loop
ganIterations = 0
train_step = 0
for epoch in range(niter):
    print(epoch)
    epoch_cost = 0
    test_loss = 0
    
    train_group, test_group = index_group(batchSize, train_num, test_num)
    for batch_id in train_group:
        
        input_cpu, target_cpu = data_loading(batch_id, path_train_data)
#         input_cpu = input_cpu[:,0:3,:,:]
#         target_cpu = target_cpu[:,0:3,:,:]
        input_cpu = torch.tensor(input_cpu, dtype = torch.float)
        target_cpu = torch.tensor(target_cpu, dtype = torch.float)
        
        batch_size = target_cpu.size(0)

        target_cpu, input_cpu = target_cpu.cuda(), input_cpu.cuda()
        # NOTE
        # default: imgA: target, imgB: input
        target.data.resize_as_(target_cpu).copy_(target_cpu)
        input_.data.resize_as_(input_cpu).copy_(input_cpu)
        input_noise.normal_(0, 0.02)
        input_.add_(input_noise) 

        # max_D first
        for p in netD.parameters(): 
            p.requires_grad = True 
        netD.zero_grad()
        real_label = uniform(0,0.1)
        fake_label = uniform(0.9,1.0)
        # NOTE: compute L_cGAN in eq.(2)
        label_d.data.resize_((batch_size, 1, sizePatchGAN, sizePatchGAN)).fill_(real_label)
        output = netD(torch.cat([target, input_], 1)) # conditional
        errD_real = criterionBCE(output, label_d)
        errD_real.backward()
        D_x = output.data.mean()
#         ae_output_ = netG.Encoder(input_)
#         latent_noise.normal_(0, 0.01)
#         ae_output_.add_(latent_noise)
#         print(ae_output_.shape)
        x_hat = netG.forward(input_)
        fake = x_hat.detach()
        label_d.data.fill_(fake_label)
        output = netD(torch.cat([fake, input_], 1)) # conditional
        errD_fake = criterionBCE(output, label_d)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step() # update parameters

        # prevent computing gradients of weights in Discriminator
        for p in netD.parameters(): 
            p.requires_grad = False
        netG.zero_grad() # start to update G

        # compute L_L1 (eq.(4) in the paper
        L_img_ = criterionCAE(x_hat, target)
        L_img = lambdaIMG * L_img_
        if lambdaIMG != 0: 
            L_img.backward(retain_graph=True)

        # add more weights on the channel part
        x_hatp = x_hat * channel_map
        targetp = target * channel_map
        L_img_p = criterionCAEP(x_hatp, targetp)
        L_imgp = lambdaIMGp * L_img_p
        if lambdaIMGp != 0: 
            L_imgp.backward(retain_graph=True)

        # compute L_cGAN (eq.(2) in the paper
        label_d.data.fill_(real_label)
        output = netD(torch.cat([x_hat, input_], 1))
        errG_ = criterionBCE(output, label_d)
        errG = lambdaGAN * errG_ 
        if lambdaGAN != 0:
            errG.backward()
        D_G_z2 = output.data.mean()

        optimizerG.step()
        
        epoch_cost = epoch_cost + L_img_.item()
    
    # apply the test data
    with torch.no_grad():
        test_inp, test_op = data_loading(test_group, path_test_data)
#         test_inp = test_inp[:,0:3,:,:]
#         test_op = test_op[:,0:3,:,:]
        test_inp = torch.tensor(test_inp, dtype = torch.float)
        test_op = torch.tensor(test_op, dtype = torch.float)
        test_inp = test_inp.cuda()
        test_op = test_op.cuda()
#         temp = netG.Encoder(test_inp)
        test_prediction = netG.forward(test_inp)
        temp_test_loss = criterionCAE(test_prediction, test_op)
        
#         temp_test_loss = loss_function(test_prediction, test_op)
        test_loss = test_loss + temp_test_loss.item()
    
    # print the loss value after every several steps of training
    if print_cost == True and epoch % 1 == 0:
        print ("Cost after epoch %i: %f" % (epoch+1, epoch_cost))
        print ("Cost of test after epoch %i: %f" % (epoch+1, test_loss))
        
        test_prediction = test_prediction.cpu()
        test_prediction = np.array(test_prediction)
        save_fig(test_prediction[0,0,:,:], path_save + str(epoch) + '_prediction.jpg')
        
        test_op = test_op.cpu()
        test_op = np.array(test_op)
        save_fig(test_op[0,0,:,:], path_save + str(epoch) + '_label.jpg')
            
    if print_cost == True and epoch % 1 == 0:
        costs.append(epoch_cost)
        test_costs.append(test_loss)
    # save the model result after every several steps
  # do checkpointing
#     if epoch >= 50 and epoch % 100 == 0:
#         torch.save(netG.state_dict(), path_save + str(epoch) + '_modelG.pth')
#         torch.save(netD.state_dict(), path_save + str(epoch) + '_modelD.pth')

0


FileNotFoundError: [Errno 2] No such file or directory: '../../Sync/Kun/30_min_ele/non_ele_whole/train/1902_inp.npy'